In [ ]:
#| default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Core

HierarchicalForecast contains pure Python implementations of hierarchical reconciliation methods as well as a `core.HierarchicalReconciliation` wrapper class that enables easy interaction with these methods through pandas DataFrames containing the hierarchical time series and the base predictions.

The `core.HierarchicalReconciliation` reconciliation class operates with the hierarchical time series pd.DataFrame `Y_df`, the base predictions pd.DataFrame `Y_hat_df`, the aggregation constraints matrix `S`. For more information on the creation of aggregation constraints matrix see the utils [aggregation method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br><br>

In [ ]:
#| export
import copy
import re
import reprlib
import time

from hierarchicalforecast.methods import HReconciler
from inspect import signature
from narwhals.typing import Frame, FrameT
from scipy.stats import norm
from scipy import sparse
from typing import Dict, List, Optional

import narwhals as nw
import numpy as np

In [ ]:
#| hide
import os

from fastcore.test import test_close, test_eq, test_fail
from nbdev.showdoc import show_doc
import pandas as pd

In [ ]:
#| hide
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
#| exporti
def _build_fn_name(fn) -> str:
    fn_name = type(fn).__name__
    func_params = fn.__dict__

    # Take default parameter out of names
    args_to_remove = ['insample', 'num_threads']
    if not func_params.get('nonnegative', False):
        args_to_remove.append('nonnegative')

    if fn_name == 'MinTrace' and \
        func_params['method']=='mint_shrink':
        if func_params['mint_shr_ridge'] == 2e-8:
            args_to_remove.append('mint_shr_ridge')

    func_params = [f'{name}-{value}' for name, value in func_params.items() if name not in args_to_remove]
    if func_params:
        fn_name += '_' + '_'.join(func_params)
    return fn_name

In [ ]:
#| hide
# test fn name
from hierarchicalforecast.methods import BottomUp, MinTrace

In [ ]:
#| hide
test_eq(_build_fn_name(BottomUp()), 'BottomUp')
test_eq(
    _build_fn_name(MinTrace(method='ols')), 
    'MinTrace_method-ols'
)
test_eq(
    _build_fn_name(MinTrace(method='ols', nonnegative=True)), 
    'MinTrace_method-ols_nonnegative-True'
)
test_eq(
    _build_fn_name(MinTrace(method='mint_shr')), 
    'MinTrace_method-mint_shr'
)

# core.HierarchicalReconciliation

In [ ]:
#| exporti
def _reverse_engineer_sigmah(Y_hat_df: Frame, 
                             y_hat: np.ndarray, 
                             model_name: str,
                             id_col: str = "unique_id",
                             time_col: str = "ds",
                             target_col: str = "y") -> np.ndarray:
    """
    This function assumes that the model creates prediction intervals
    under a normality with the following the Equation:
    $\hat{y}_{t+h} + c \hat{sigma}_{h}$

    In the future, we might deprecate this function in favor of a 
    direct usage of an estimated $\hat{sigma}_{h}$
    """

    drop_cols = [time_col]
    if target_col in Y_hat_df.columns:
        drop_cols.append(target_col)
    if model_name+'-median' in Y_hat_df.columns:
        drop_cols.append(model_name+'-median')
    model_names = [c for c in Y_hat_df.columns if c not in drop_cols]
    pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name)]
    pi_model_name = [pi_name for pi_name in pi_model_names if model_name in pi_name]
    pi = len(pi_model_name) > 0

    n_series = Y_hat_df[id_col].n_unique()

    if not pi:
        raise ValueError(f'Please include `{model_name}` prediction intervals in `Y_hat_df`')

    pi_col = pi_model_name[0]
    sign = -1 if 'lo' in pi_col else 1
    level_cols = re.findall('[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', pi_col)
    level_col = float(level_cols[-1])
    z = norm.ppf(0.5 + level_col / 200)
    sigmah = Y_hat_df[pi_col].to_numpy().reshape(n_series,-1)
    sigmah = sign * (sigmah - y_hat) / z

    return sigmah

In [ ]:
#| export
class HierarchicalReconciliation:
    """Hierarchical Reconciliation Class.

    The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
    HierarchicaForecast methods for a collection of time series and base predictions stored in 
    pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
    y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
    example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

    **Parameters:**<br>
    `reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

    **References:**<br>
    [Rob J. Hyndman and George Athanasopoulos (2018). \"Forecasting principles and practice, Hierarchical and Grouped Series\".](https://otexts.com/fpp3/hierarchical.html)
    """
    def __init__(self,
                 reconcilers: List[HReconciler]):
        self.reconcilers = reconcilers
        self.orig_reconcilers = copy.deepcopy(reconcilers) # TODO: elegant solution
        self.insample = any([method.insample for method in reconcilers])
    
    def _prepare_fit(self,
                     Y_hat_nw: Frame,
                     S_nw: Frame,
                     Y_nw: Optional[Frame],
                     tags: Dict[str, np.ndarray],
                     level: Optional[List[int]] = None,
                     intervals_method: str = 'normality',
                     id_col: str = "unique_id",
                     time_col: str = "ds", 
                     target_col: str = "y",                      
                     ) -> tuple[FrameT, FrameT, FrameT, List[str]]:
        """
        Performs preliminary wrangling and protections
        """
        Y_hat_nw_cols = Y_hat_nw.columns
        S_nw_cols = S_nw.columns

        #-------------------------------- Match Y_hat/Y/S index order --------------------------------#
        # TODO: This is now a bit slow as we always sort.
        S_nw = S_nw.with_columns(**{f"{id_col}_id": np.arange(len(S_nw))})

        Y_hat_nw = Y_hat_nw.join(S_nw[[id_col, f"{id_col}_id"]], on=id_col, how='left')
        Y_hat_nw = Y_hat_nw.sort(by=[f"{id_col}_id", time_col])
        Y_hat_nw = Y_hat_nw[Y_hat_nw_cols]

        if Y_nw is not None:
            Y_nw_cols = Y_nw.columns
            Y_nw = Y_nw.join(S_nw[[id_col, f"{id_col}_id"]], on=id_col, how='left')
            Y_nw = Y_nw.sort(by=[f"{id_col}_id", time_col])
            Y_nw = Y_nw[Y_nw_cols]

        S_nw = S_nw[S_nw_cols]

        #----------------------------------- Check Input's Validity ----------------------------------#

        # Check input's validity
        if intervals_method not in ['normality', 'bootstrap', 'permbu']:
            raise ValueError(f'Unknown interval method: {intervals_method}')

        # TODO: this logic should be method specific
        if (self.insample or (intervals_method in ['bootstrap', 'permbu'])):
            if Y_nw is None:
                raise Exception('You need to provide `Y_df`.')
        
        # Protect level list
        if (level is not None):
            level_outside_domain = not all(0 <= x < 100 for x in level)
            if level_outside_domain and (intervals_method in ['normality', 'permbu']):
                raise ValueError("Level must be a list containing floating values in the interval [0, 100).")

        # Declare output names
        model_names = [col for col in Y_hat_nw.columns if col not in [id_col, time_col, target_col]]

        # Ensure numeric columns
        for model in model_names:
            if not Y_hat_nw.schema[model].is_numeric():
                raise ValueError(f"Column `{model}` in `Y_hat_df` contains non-numeric values. Make sure no column in `Y_hat_df` contains non-numeric values.")
            if Y_hat_nw[model].is_null().any():
                raise ValueError(f"Column `{model}` in `Y_hat_df` contains null values. Make sure no column in `Y_hat_df` contains null values.")

        # TODO: Complete y_hat_insample protection
        model_names = [name for name in model_names if not ('-lo' in name or '-hi' in name or '-median' in name)]        
        if intervals_method in ['bootstrap', 'permbu'] and Y_nw is not None:
            missing_models = set(model_names) - set(Y_nw.columns)
            if len(missing_models) > 0:
                raise ValueError(f"Check `Y_df` columns, {reprlib.repr(missing_models)} must be in `Y_df` columns.")

        # Assert S is an identity matrix at the bottom
        S_nw_cols.remove(id_col)
        if not np.allclose(S_nw[S_nw_cols][-len(S_nw_cols):], np.eye(len(S_nw_cols))):
            raise ValueError(f"The bottom {S_nw.shape[1]}x{S_nw.shape[1]} part of S must be an identity matrix.")

        # Check Y_hat_df\S_df series difference
        # TODO: this logic should be method specific
        S_diff = set(S_nw[id_col]) - set(Y_hat_nw[id_col])
        Y_hat_diff = set(Y_hat_nw[id_col]) - set(S_nw[id_col])
        if S_diff:
            raise ValueError(f'There are unique_ids in S_df that are not in Y_hat_df: {reprlib.repr(S_diff)}')
        if Y_hat_diff:
            raise ValueError(f'There are unique_ids in Y_hat_df that are not in S_df: {reprlib.repr(Y_hat_diff)}')

        if Y_nw is not None:
            Y_diff = set(Y_nw[id_col]) - set(Y_hat_nw[id_col])
            Y_hat_diff = set(Y_hat_nw[id_col]) - set(Y_nw[id_col])
            if Y_diff:
                raise ValueError(f'There are unique_ids in Y_df that are not in Y_hat_df: {reprlib.repr(Y_diff)}')
            if Y_hat_diff:
                raise ValueError(f'There are unique_ids in Y_hat_df that are not in Y_df: {reprlib.repr(Y_hat_diff)}')

        # Same Y_hat_df/S_df/Y_df's unique_ids. Order is guaranteed by sorting.
        # TODO: this logic should be method specific
        unique_ids = Y_hat_nw[id_col].unique().to_numpy()
        S_nw = S_nw.filter(nw.col(id_col).is_in(unique_ids))

        return Y_hat_nw, S_nw, Y_nw, model_names

    def _prepare_Y(self, 
                          Y_nw: Frame, 
                          S_nw: Frame, 
                          is_balanced: bool = True,
                          id_col: str = "unique_id",
                          time_col: str = "ds", 
                          target_col: str = "y", 
                          ) -> np.ndarray:
        """
        Prepare Y data.
        """
        if is_balanced:
            Y = Y_nw[target_col].to_numpy().reshape(len(S_nw), -1)
        else:
            Y_pivot = Y_nw.pivot(on=time_col, index=id_col, values=target_col, sort_columns=True).sort(by=id_col)
            Y_pivot_cols_ex_id_col = Y_pivot.columns
            Y_pivot_cols_ex_id_col.remove(id_col)

            # TODO: check if this is the best way to do it - it's reasonably fast to ensure Y_pivot has same order as S_nw
            pos_in_Y = np.searchsorted(Y_pivot[id_col].to_numpy(), S_nw[id_col].to_numpy())
            Y_pivot = Y_pivot.select(nw.col(Y_pivot_cols_ex_id_col))
            Y_pivot = Y_pivot[pos_in_Y]
            Y = Y_pivot.to_numpy()

        # TODO: the result is a Fortran contiguous array, see if we can avoid the below copy (I don't think so)
        Y = np.ascontiguousarray(Y, dtype=np.float64)
        return Y


    def reconcile(self, 
                  Y_hat_df: Frame,
                  S: Frame,
                  tags: Dict[str, np.ndarray],
                  Y_df: Optional[Frame] = None,
                  level: Optional[List[int]] = None,
                  intervals_method: str = 'normality',
                  num_samples: int = -1,
                  seed: int = 0,
                  is_balanced: bool = False,
                  id_col: str = "unique_id",
                  time_col: str = "ds", 
                  target_col: str = "y",                   
        ) -> FrameT:
        """Hierarchical Reconciliation Method.

        The `reconcile` method is analogous to SKLearn `fit_predict` method, it 
        applies different reconciliation techniques instantiated in the `reconcilers` list.

        Most reconciliation methods can be described by the following convenient 
        linear algebra notation:

        $$\\tilde{\mathbf{y}}_{[a,b],\\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\\tau}$$

        where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
        the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
        reconciliation methods. The reconciled predictions are $\\tilde{\mathbf{y}}_{[a,b],\\tau}$, and the 
        base predictions $\hat{\mathbf{y}}_{[a,b],\\tau}$.

        **Parameters:**<br>
        `Y_hat_df`: DataFrame, base forecasts with columns ['unique_id', 'ds'] and models to reconcile.<br>
        `Y_df`: DataFrame, training set of base time series with columns `['unique_id', 'ds', 'y']`.<br>
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: positive float list [0,100), confidence levels for prediction intervals.<br>
        `intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
        `num_samples`: int=-1, if positive return that many probabilistic coherent samples.
        `seed`: int=0, random seed for numpy generator's replicability.<br>
        `is_balanced`: bool=False, wether `Y_df` is balanced, set it to True to speed things up if `Y_df` is balanced.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>

        **Returns:**<br>
        `Y_tilde_df`: DataFrame, with reconciled predictions.
        """
        # To Narwhals
        Y_hat_nw = nw.from_native(Y_hat_df)
        S_nw = nw.from_native(S)
        if Y_df is not None:
            Y_nw = nw.from_native(Y_df)
        else:
            Y_nw = None

        # Check input's validity and sort dataframes
        Y_hat_nw, S_nw, Y_nw, self.model_names = \
                    self._prepare_fit(Y_hat_nw=Y_hat_nw,
                                      S_nw=S_nw,
                                      Y_nw=Y_nw,
                                      tags=tags,
                                      level=level,
                                      intervals_method=intervals_method,
                                      id_col=id_col,
                                      time_col=time_col,
                                      target_col=target_col,                                     
                                      )

        # Initialize reconciler arguments
        reconciler_args = dict(
            idx_bottom=np.arange(len(S_nw))[-S_nw.shape[1]:],
            tags={key: S_nw.with_columns(nw.col(id_col).is_in(val).alias("in_cols"))["in_cols"].to_numpy().nonzero()[0] for key, val in tags.items()},
        )

        any_sparse = any([method.is_sparse_method for method in self.reconcilers])
        S_nw_cols_ex_id_col = S_nw.columns
        S_nw_cols_ex_id_col.remove(id_col)
        if any_sparse:
            if not nw.dependencies.is_pandas_dataframe(Y_hat_df) or not nw.dependencies.is_pandas_dataframe(S):
                raise ValueError("You have one or more sparse reconciliation methods. Please convert `S` and `Y_hat_df` to a pandas DataFrame.")
            try:
                S_for_sparse = sparse.csr_matrix(S_nw.select(nw.col(S_nw_cols_ex_id_col)).to_native().sparse.to_coo())                
            except AttributeError:
                S_for_sparse = sparse.csr_matrix(S_nw.select(nw.col(S_nw_cols_ex_id_col)).to_numpy().astype(np.float64, copy=False))

        if Y_nw is not None:
            if any_sparse and not nw.dependencies.is_pandas_dataframe(Y_df):
                raise ValueError("You have one or more sparse reconciliation methods. Please convert `Y_df` to a pandas DataFrame.")      
            y_insample = self._prepare_Y(Y_nw=Y_nw, 
                                         S_nw=S_nw, 
                                         is_balanced=is_balanced, 
                                         id_col=id_col, 
                                         time_col=time_col, 
                                         target_col=target_col)     
            reconciler_args['y_insample'] = y_insample

        Y_tilde_nw = nw.maybe_reset_index(Y_hat_nw.clone())
        self.execution_times = {}
        self.level_names = {}
        self.sample_names = {}
        for reconciler, name_copy in zip(self.reconcilers, self.orig_reconcilers):
            reconcile_fn_name = _build_fn_name(name_copy)

            if reconciler.is_sparse_method:
                reconciler_args["S"] = S_for_sparse
            else:
                reconciler_args["S"] = S_nw.select(nw.col(S_nw_cols_ex_id_col))\
                                           .to_numpy()\
                                           .astype(np.float64, copy=False)

            has_fitted = 'y_hat_insample' in signature(reconciler.fit_predict).parameters
            has_level = 'level' in signature(reconciler.fit_predict).parameters

            for model_name in self.model_names:
                start = time.time()
                recmodel_name = f'{model_name}/{reconcile_fn_name}'

                # TODO: the below should be method specific
                y_hat = self._prepare_Y(Y_nw=Y_hat_nw[[id_col, time_col, model_name]], 
                                        S_nw=S_nw, 
                                        is_balanced=True, 
                                        id_col=id_col, 
                                        time_col=time_col, 
                                        target_col=model_name)
                reconciler_args['y_hat'] = y_hat

                if (self.insample and has_fitted) or intervals_method in ['bootstrap', 'permbu']:
                    y_hat_insample = self._prepare_Y(Y_nw=Y_nw[[id_col, time_col, model_name]], 
                                         S_nw=S_nw, 
                                         is_balanced=is_balanced, 
                                         id_col=id_col, 
                                         time_col=time_col, 
                                         target_col=model_name)   
                    reconciler_args['y_hat_insample'] = y_hat_insample

                if has_level and (level is not None):
                    if intervals_method in ['normality', 'permbu']:
                        sigmah = _reverse_engineer_sigmah(Y_hat_df=Y_hat_nw,
                                    y_hat=y_hat, model_name=model_name)
                        reconciler_args['sigmah'] = sigmah

                    reconciler_args['intervals_method'] = intervals_method
                    reconciler_args['num_samples'] = 200 # TODO: solve duplicated num_samples
                    reconciler_args['seed'] = seed

                # Mean and Probabilistic reconciliation
                kwargs_ls = [key for key in signature(reconciler.fit_predict).parameters if key in reconciler_args.keys()]
                kwargs = {key: reconciler_args[key] for key in kwargs_ls}
                
                if (level is not None) and (num_samples > 0):
                    # Store reconciler's memory to generate samples
                    reconciler = reconciler.fit(**kwargs)
                    fcsts_model = reconciler.predict(S=reconciler_args['S'], 
                                                     y_hat=reconciler_args['y_hat'], level=level)
                else:
                    # Memory efficient reconciler's fit_predict
                    fcsts_model = reconciler(**kwargs, level=level)

                # Parse final outputs
                Y_tilde_nw = Y_tilde_nw.with_columns(**{recmodel_name: fcsts_model["mean"].flatten()})
                
                if intervals_method in ['bootstrap', 'normality', 'permbu'] and level is not None:
                    level.sort()
                    lo_names = [f'{recmodel_name}-lo-{lv}' for lv in reversed(level)]
                    hi_names = [f'{recmodel_name}-hi-{lv}' for lv in level]
                    self.level_names[recmodel_name] = lo_names + hi_names
                    sorted_quantiles = np.reshape(fcsts_model['quantiles'], (len(Y_tilde_nw), -1))
                    y_tilde = dict(zip(self.level_names[recmodel_name], sorted_quantiles.T))
                    Y_tilde_nw = Y_tilde_nw.with_columns(**y_tilde)

                    if num_samples > 0:
                        samples = reconciler.sample(num_samples=num_samples)
                        self.sample_names[recmodel_name] = [f'{recmodel_name}-sample-{i}' for i in range(num_samples)]
                        samples = np.reshape(samples, (len(Y_tilde_nw),-1))        
                        y_tilde = dict(zip(self.sample_names[recmodel_name], samples.T))
                        Y_tilde_nw = Y_tilde_nw.with_columns(**y_tilde)
                      

                end = time.time()
                self.execution_times[f'{model_name}/{reconcile_fn_name}'] = (end - start)

        Y_tilde_df = Y_tilde_nw.to_native()

        return Y_tilde_df

    def bootstrap_reconcile(self,
                            Y_hat_df: Frame,
                            S_df: Frame,
                            tags: Dict[str, np.ndarray],
                            Y_df: Optional[Frame] = None,
                            level: Optional[List[int]] = None,
                            intervals_method: str = 'normality',
                            num_samples: int = -1,
                            num_seeds: int = 1,
                            id_col: str = "unique_id",
                            time_col: str = "ds", 
                            target_col: str = "y",                                 
                            ) -> FrameT:
        """Bootstraped Hierarchical Reconciliation Method.

        Applies N times, based on different random seeds, the `reconcile` method 
        for the different reconciliation techniques instantiated in the `reconcilers` list. 

        **Parameters:**<br>
        `Y_hat_df`: DataFrame, base forecasts with columns ['unique_id', 'ds'] and models to reconcile.<br>
        `Y_df`: DataFrame, training set of base time series with columns `['unique_id', 'ds', 'y']`.<br>
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: positive float list [0,100), confidence levels for prediction intervals.<br>
        `intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
        `num_samples`: int=-1, if positive return that many probabilistic coherent samples.
        `num_seeds`: int=1, random seed for numpy generator's replicability.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>        

        **Returns:**<br>
        `Y_bootstrap_df`: DataFrame, with bootstraped reconciled predictions.
        """
        # Bootstrap reconciled predictions
        Y_tilde_list = []
        for seed in range(num_seeds):
            Y_tilde_df = self.reconcile(Y_hat_df=Y_hat_df,
                                        S=S_df,
                                        tags=tags,
                                        Y_df=Y_df,
                                        level=level,
                                        intervals_method=intervals_method,
                                        num_samples=num_samples,
                                        seed=seed,
                                        id_col=id_col,
                                        time_col=time_col,
                                        target_col=target_col,
                                        )
            Y_tilde_nw = nw.from_native(Y_tilde_df)
            Y_tilde_nw = Y_tilde_nw.with_columns(nw.lit(seed).alias('seed'))

            # TODO: fix broken recmodel_names
            if seed==0:
                first_columns = Y_tilde_nw.columns
            Y_tilde_nw = Y_tilde_nw.rename({col: first_columns[i] for i, col in enumerate(first_columns)})
            Y_tilde_list.append(Y_tilde_nw)

        Y_bootstrap_nw = nw.concat(Y_tilde_list, how="vertical")
        Y_bootstrap_df = Y_bootstrap_nw.to_native()

        return Y_bootstrap_df

In [ ]:
show_doc(HierarchicalReconciliation, 
         name='init', title_level=3)

In [ ]:
show_doc(HierarchicalReconciliation)

In [ ]:
show_doc(HierarchicalReconciliation.reconcile, name='reconcile', title_level=3)

In [ ]:
show_doc(HierarchicalReconciliation.bootstrap_reconcile, name='bootstrap_reconcile', title_level=3)

In [ ]:
#| hide
from hierarchicalforecast.methods import (
    BottomUp, TopDown, MiddleOut, MinTrace, ERM,
)
from hierarchicalforecast.utils import aggregate

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
df['ds'] = df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
df['ds'] = pd.to_datetime(df['ds'])

# non strictly hierarchical structure
hierS_grouped_df = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped_df, tags_grouped = aggregate(df, hierS_grouped_df)
hier_strict_df, S_strict, tags_strict = aggregate(df, hiers_strictly)

# check categorical input produces same output
df2 = df.copy()
for col in ['Country', 'State', 'Purpose', 'Region']:
    df2[col] = df2[col].astype('category')

for spec in [hierS_grouped_df, hiers_strictly]:
    Y_orig, S_orig, tags_orig = aggregate(df, spec)
    Y_cat, S_cat, tags_cat = aggregate(df2, spec)
    pd.testing.assert_frame_equal(Y_cat, Y_orig)
    pd.testing.assert_frame_equal(S_cat, S_orig)
    assert all(np.array_equal(tags_orig[k], tags_cat[k]) for k in tags_orig.keys())

In [ ]:
#| hide
# polars
import polars as pl
import polars.testing as pltest

In [ ]:
#| hide
# polars
df_pl = pl.DataFrame(df)

# getting df
hier_grouped_df_pl, S_grouped_df_pl, tags_grouped_pl = aggregate(df_pl, hierS_grouped_df)
hier_strict_df_pl, S_strict_pl, tags_strict_pl = aggregate(df_pl, hiers_strictly)

# check categorical input produces same output
df2_pl = df_pl.clone()
for col in ['Country', 'State', 'Purpose', 'Region']:
    df2_pl = df2_pl.with_columns(pl.col(col).cast(pl.Categorical))

for spec in [hierS_grouped_df, hiers_strictly]:
    Y_orig_pl, S_orig_pl, tags_orig_pl = aggregate(df_pl, spec)
    Y_cat_pl, S_cat_pl, tags_cat_pl = aggregate(df2_pl, spec)
    pltest.assert_frame_equal(Y_cat_pl, Y_orig_pl)
    pltest.assert_frame_equal(S_cat_pl, S_orig_pl)
    assert all(np.array_equal(tags_orig_pl[k], tags_cat_pl[k]) for k in tags_orig_pl.keys())


In [ ]:
#| hide
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_hat_df = hier_grouped_df.groupby('unique_id').tail(12)
ds_h = hier_grouped_hat_df['ds'].unique()
hier_grouped_df_filtered = hier_grouped_df.query('~(ds in @ds_h)').copy()
# adding noise to `y_model` to avoid perfect fited values
hier_grouped_df_filtered['y_model'] += np.random.uniform(-1, 1, len(hier_grouped_df_filtered))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df, 
                            Y_df=hier_grouped_df_filtered, 
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=["unique_id", "ds", "y"]).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# polars
hier_grouped_hat_df_pl = pl.from_pandas(hier_grouped_hat_df)
hier_grouped_df_filtered_pl = pl.from_pandas(hier_grouped_df_filtered)
S_grouped_df_pl = pl.from_pandas(S_grouped_df)

reconciled_pl = hrec.reconcile(Y_hat_df=hier_grouped_hat_df_pl, 
                            Y_df=hier_grouped_df_filtered_pl, 
                            S=S_grouped_df_pl, 
                            tags=tags_grouped)

for model in reconciled_pl.drop(["unique_id", "ds", "y"]).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled_pl['y'], reconciled_pl[model], eps=eps)

In [ ]:
#| hide
# test incorrect Y_hat_df datatypes
hier_grouped_hat_df_nan = hier_grouped_hat_df.copy()
hier_grouped_hat_df_idx_changed = hier_grouped_hat_df_nan.query("unique_id == 'Australia'").index
hier_grouped_hat_df_nan.loc[hier_grouped_hat_df_idx_changed, 'y_model'] = float('nan')
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_nan, S_grouped_df, tags_grouped, hier_grouped_df),
)

hier_grouped_hat_df_none = hier_grouped_hat_df.copy()
hier_grouped_hat_df_idx_changed = hier_grouped_hat_df_none.query("unique_id == 'Australia'").index
hier_grouped_hat_df_none.loc[hier_grouped_hat_df_idx_changed, 'y_model'] = None
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_none, S_grouped_df, tags_grouped, hier_grouped_df),
)

hier_grouped_hat_df_str = hier_grouped_hat_df.copy()
hier_grouped_hat_df_str['y_model'] = hier_grouped_hat_df_str['y_model'].astype(str)
test_fail(
    hrec.reconcile,
    contains='numeric values',
    args=(hier_grouped_hat_df_str, S_grouped_df, tags_grouped, hier_grouped_df),
)

In [ ]:
#| hide
# polars
# test incorrect Y_hat_df datatypes
hier_grouped_hat_df_nan_pl = pl.from_pandas(hier_grouped_hat_df_nan)
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_nan_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl),
)

hier_grouped_hat_df_none_pl = pl.from_pandas(hier_grouped_hat_df_none)
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_none_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl),
)

hier_grouped_hat_df_str_pl = pl.from_pandas(hier_grouped_hat_df_str)
test_fail(
    hrec.reconcile,
    contains='numeric values',
    args=(hier_grouped_hat_df_str_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl),
)

In [ ]:
#| hide
# test expected error
# different series S and Y_hat_df
drop_idx = hier_grouped_hat_df.query("unique_id == 'Australia'").index
test_fail(
    hrec.reconcile,
    contains='There are unique_ids in S_df that are not in Y_hat_df',
    args=(hier_grouped_hat_df.drop(index=drop_idx), S_grouped_df, tags_grouped, hier_grouped_df),
    
)

drop_idx = S_grouped_df.query("unique_id == 'Australia'").index
test_fail(
    hrec.reconcile,
    contains='There are unique_ids in Y_hat_df that are not in S_df',
    args=(hier_grouped_hat_df, S_grouped_df.drop(index=drop_idx), tags_grouped, hier_grouped_df),
)

drop_idx = hier_grouped_df.query("unique_id == 'Australia'").index
test_fail(
    hrec.reconcile,
    contains='There are unique_ids in Y_hat_df that are not in Y_df',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df.drop(index=drop_idx)),   
)

In [ ]:
#| hide
# polars
# test expected error
# different series S and Y_hat_df
test_fail(
    hrec.reconcile,
    contains='There are unique_ids in S_df that are not in Y_hat_df',
    args=(hier_grouped_hat_df_pl.filter(pl.col("unique_id") != "Australia"), S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl),
)

test_fail(
    hrec.reconcile,
    contains='There are unique_ids in Y_hat_df that are not in S_df',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl.filter(pl.col("unique_id") != "Australia"), tags_grouped_pl, hier_grouped_df_pl),
)

test_fail(
    hrec.reconcile,
    contains='There are unique_ids in Y_hat_df that are not in Y_df',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl.filter(pl.col("unique_id") != "Australia")),   
)

In [ ]:
# | hide
# test expected error
# different columns Y_df and Y_hat_df
hrec = HierarchicalReconciliation(
            reconcilers=[ERM(method='reg_bu', lambda_reg=100)])
test_fail(
    hrec.reconcile,
    contains='Please include ',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, 
          hier_grouped_df, [80], 'permbu'), # permbu needs y_hat_insample
)

In [ ]:
# | hide
# polars
# test expected error
# different columns Y_df and Y_hat_df
hier_grouped_hat_df_pl = pl.from_pandas(hier_grouped_hat_df)
hier_grouped_df_pl = pl.from_pandas(hier_grouped_df)
S_grouped_df_pl = pl.from_pandas(S_grouped_df)

test_fail(
    hrec.reconcile,
    contains='Please include ',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl, 
          hier_grouped_df_pl, [80], 'permbu'), # permbu needs y_hat_insample
)

In [ ]:
#| hide
# test reconcile method without insample
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df,
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y', 'unique_id']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# polars
# test reconcile method without insample
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df_pl,
                            S=S_grouped_df_pl, 
                            tags=tags_grouped_pl)
for model in reconciled.drop(['ds', 'y', 'unique_id']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# top down should break
# with non strictly hierarchical structures
hrec = HierarchicalReconciliation([TopDown(method='average_proportions')])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  hier_grouped_df,)
)

In [ ]:
#| hide
# polars
# top down should break
# with non strictly hierarchical structures
hrec = HierarchicalReconciliation([TopDown(method='average_proportions')])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl,  hier_grouped_df_pl,)
)

In [ ]:
#| hide
# methods should work with strictly hierarchical structures
hier_strict_df['y_model'] = hier_strict_df['y']
# we should be able to recover y using the methods
hier_strict_df_h = hier_strict_df.groupby('unique_id').tail(12)
ds_h = hier_strict_df_h['ds'].unique()
hier_strict_df = hier_strict_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_strict_df['y_model'] += np.random.uniform(-1, 1, len(hier_strict_df))

middle_out_level = 'Country/State'
# hierarchical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # top down doesnt recover the original y
    # but it should recover the total level
    TopDown(method='forecast_proportions'),
    TopDown(method='average_proportions'),
    TopDown(method='proportion_averages'),
    # middle out doesnt recover the original y
    # but it should recover the total level
    MiddleOut(middle_level=middle_out_level, top_down_method='forecast_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='average_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='proportion_averages'),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict
)
for model in reconciled.drop(columns=['ds', 'y', 'unique_id']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    if 'TopDown' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict['Country']
        test_close(reconciled[["unique_id", "y"]].query("unique_id == @total_tag[0]")["y"], 
                   reconciled[["unique_id", model]].query("unique_id == @total_tag[0]")[model], 1e-2)
    elif 'MiddleOut' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict[middle_out_level]
        test_close(reconciled[["unique_id", "y"]].query("unique_id == @total_tag[0]")["y"], 
                   reconciled[["unique_id", model]].query("unique_id == @total_tag[0]")[model], 1e-2)
    else:
        test_close(reconciled['y'], reconciled[model], eps)

In [ ]:
#| hide
# polars
# methods should work with strictly hierarchical structures
hier_strict_df_pl = hier_strict_df_pl.with_columns(hier_strict_df_pl['y'].alias('y_model'))
# we should be able to recover y using the methods
hier_strict_df_h_pl = hier_strict_df_pl.group_by('unique_id').tail(12)
ds_h = set(hier_strict_df_h_pl['ds'])
hier_strict_df_pl = hier_strict_df_pl.filter(~pl.col("ds").is_in(ds_h))
#adding noise to `y_model` to avoid perfect fited values
hier_strict_df_pl = hier_strict_df_pl.with_columns(pl.col('y_model') + np.random.uniform(-1, 1, len(hier_strict_df_pl)))

middle_out_level = 'Country/State'
# hierarchical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # top down doesnt recover the original y
    # but it should recover the total level
    TopDown(method='forecast_proportions'),
    TopDown(method='average_proportions'),
    TopDown(method='proportion_averages'),
    # middle out doesnt recover the original y
    # but it should recover the total level
    MiddleOut(middle_level=middle_out_level, top_down_method='forecast_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='average_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='proportion_averages'),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled_pl = hrec.reconcile(
    Y_hat_df=hier_strict_df_h_pl, 
    Y_df=hier_strict_df_pl, 
    S=S_strict_pl, 
    tags=tags_strict_pl
)
for model in reconciled_pl.drop(['ds', 'y', 'unique_id']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    if 'TopDown' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled_pl['y'], reconciled_pl[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled_pl['y'], reconciled_pl[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict['Country']
        test_close(reconciled_pl[["unique_id", "y"]].filter(pl.col("unique_id") == total_tag[0])["y"], 
                   reconciled_pl[["unique_id", model]].filter(pl.col("unique_id") == total_tag[0])[model], 1e-2)
    elif 'MiddleOut' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled_pl['y'], reconciled_pl[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled_pl['y'], reconciled_pl[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict[middle_out_level]
        test_close(reconciled_pl[["unique_id", "y"]].filter(pl.col("unique_id") == total_tag[0])["y"], 
                   reconciled_pl[["unique_id", model]].filter(pl.col("unique_id") == total_tag[0])[model], 1e-2)
    else:
        test_close(reconciled_pl['y'], reconciled_pl[model], eps)

In [ ]:
#| hide
# test is_balanced behaviour
reconciled_balanced = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict,
    is_balanced=True,
)
test_close(reconciled.drop(columns=["unique_id", "ds"]).values, reconciled_balanced.drop(columns=["unique_id", "ds"]).values, eps=1e-10)

In [ ]:
#| hide
# polars
# test is_balanced behaviour
reconciled_balanced = hrec.reconcile(
    Y_hat_df=hier_strict_df_h_pl, 
    Y_df=hier_strict_df_pl, 
    S=S_strict_pl, 
    tags=tags_strict_pl,
    is_balanced=True,
)
test_close(reconciled_pl.drop(["unique_id", "ds"]).to_numpy(), reconciled_balanced.drop(["unique_id", "ds"]).to_numpy(), eps=1e-10)

In [ ]:
#| hide
from statsforecast import StatsForecast
from statsforecast.utils import generate_series
from statsforecast.models import RandomWalkWithDrift

In [ ]:
#| hide
# test unbalanced dataset
max_tenure = 24
dates = pd.date_range(start='2019-01-31', freq='ME', periods=max_tenure)
cohort_tenure = [24, 23, 22, 21]

ts_list = []

# Create ts for each cohort
for i in range(len(cohort_tenure)):
    ts_list.append(
        generate_series(n_series=1, freq='ME', min_length=cohort_tenure[i], max_length=cohort_tenure[i]).reset_index() \
            .assign(ult=i) \
            .assign(ds=dates[-cohort_tenure[i]:]) \
            .drop(columns=['unique_id'])
    )
df = pd.concat(ts_list, ignore_index=True)

# Create categories
df.loc[df['ult'] < 2, 'pen'] = 'a'
df.loc[df['ult'] >= 2, 'pen'] = 'b'
# Note that unique id requires strings
df['ult'] = df['ult'].astype(str)

hier_levels = [
    ['pen'],
    ['pen', 'ult'],
]
hier_df, S_df, tags = aggregate(df=df, spec=hier_levels)

train_df = hier_df.query("ds <= @pd.to_datetime('2019-12-31')")
test_df = hier_df.query("ds > @pd.to_datetime('2019-12-31')")

fcst = StatsForecast(
    models=[
        RandomWalkWithDrift(),
    ],
    freq='ME',
    n_jobs=1,
)

hrec = HierarchicalReconciliation(
    reconcilers=[
        BottomUp(),
        MinTrace(method='mint_shrink'),
    ]
)

fcst_df = fcst.forecast(df=train_df, h=12, fitted=True)
fitted_df = fcst.forecast_fitted_values()

fcst_df = hrec.reconcile(
    Y_hat_df=fcst_df,
    Y_df=fitted_df,
    S=S_df,
    tags=tags,
)

In [ ]:
#| hide
# polars
# test unbalanced dataset
df_pl = pl.from_pandas(df)
hier_df_pl, S_df_pl, tags_pl = aggregate(df=df_pl, spec=hier_levels)

train_df = hier_df_pl.filter(pl.col("ds") <= pl.lit('2019-12-31').str.to_date())
test_df = hier_df_pl.filter(pl.col("ds") > pl.lit('2019-12-31').str.to_date())

fcst = StatsForecast(
    models=[
        RandomWalkWithDrift(),
    ],
    freq='1mo',
    n_jobs=1,
)

hrec = HierarchicalReconciliation(
    reconcilers=[
        BottomUp(),
        MinTrace(method='mint_shrink'),
    ]
)

fcst_df_pl = fcst.forecast(df=train_df, h=12, fitted=True)
fitted_df = fcst.forecast_fitted_values()

fcst_df_pl = hrec.reconcile(
    Y_hat_df=fcst_df_pl,
    Y_df=fitted_df,
    S=S_df_pl,
    tags=tags,
)

# Test equivalence
pd.testing.assert_frame_equal(fcst_df, fcst_df_pl.to_pandas())

In [ ]:
#| hide
# MinTrace should break
# with extremely overfitted model, y_model==y
zero_df = hier_grouped_df.copy()
zero_df['y'] = 0
zero_df['y_model'] = 0
hrec = HierarchicalReconciliation([MinTrace(method='mint_shrink')])
test_fail(
    hrec.reconcile,
    contains='Insample residuals',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  zero_df)
)

In [ ]:
#| hide
# polars
# MinTrace should break
# with extremely overfitted model, y_model==y
zero_df_pl = pl.from_pandas(zero_df)    
hrec = HierarchicalReconciliation([MinTrace(method='mint_shrink')])
test_fail(
    hrec.reconcile,
    contains='Insample residuals',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl,  zero_df_pl)
)

In [ ]:
#| hide
#test methods that dont use residuals
#even if their signature includes
#that argument
hrec = HierarchicalReconciliation([MinTrace(method='ols')])
reconciled = hrec.reconcile(
    Y_hat_df=hier_grouped_hat_df, 
    Y_df=hier_grouped_df.drop(columns=['y_model']), 
    S=S_grouped_df, 
    tags=tags_grouped
)
for model in reconciled.drop(columns=['ds', 'y', 'unique_id']).columns:
    test_close(reconciled['y'], reconciled[model], eps=1e-1)

In [ ]:
#| hide
# polars
#test methods that dont use residuals
#even if their signature includes
#that argument
hrec = HierarchicalReconciliation([MinTrace(method='ols')])
reconciled = hrec.reconcile(
    Y_hat_df=hier_grouped_hat_df_pl, 
    Y_df=hier_grouped_df_pl.drop(['y_model']), 
    S=S_grouped_df_pl, 
    tags=tags_grouped_pl
)
for model in reconciled.drop(['ds', 'y', 'unique_id']).columns:
    test_close(reconciled['y'], reconciled[model], eps=1e-1)

In [ ]:
#| hide
# test methods with bootstrap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df, 
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='bootstrap')
total = reconciled.query("unique_id in @tags_grouped['Country/State/Region/Purpose']").groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.query("unique_id == 'Australia'")[['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# polars
# test methods with bootstrap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df_pl, 
                            Y_df=hier_grouped_df_pl, 
                            S=S_grouped_df_pl, 
                            tags=tags_grouped_pl,
                            level=[80, 90], 
                            intervals_method='bootstrap')
total = reconciled.filter(pl.col("unique_id").is_in(tags_grouped['Country/State/Region/Purpose'])).group_by('ds', maintain_order=True).sum()
pltest.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.filter(pl.col("unique_id") == 'Australia')[['ds', 'y_model/BottomUp']]
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with  normality prediction intervals
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df,
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='normality')
total = reconciled.query("unique_id in @tags_grouped['Country/State/Region/Purpose']").groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.query("unique_id == 'Australia'")[['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# polars
# test methods with  normality prediction intervals
hier_grouped_hat_df_pl = pl.from_pandas(hier_grouped_hat_df)
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df_pl,
                            Y_df=hier_grouped_df_pl, 
                            S=S_grouped_df_pl, 
                            tags=tags_grouped_pl,
                            level=[80, 90], 
                            intervals_method='normality')
total = reconciled.filter(pl.col("unique_id").is_in(tags_grouped['Country/State/Region/Purpose'])).group_by('ds', maintain_order=True).sum()
pltest.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.filter(pl.col("unique_id") == 'Australia')[['ds', 'y_model/BottomUp']]
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with PERMBU prediction intervals

# test expect error with grouped structure
# (non strictly hierarchical)
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [80, 90], 'permbu',)
)

# test PERMBU
hier_strict_df_h['y_model-lo-80'] = hier_strict_df_h['y_model'] - 1.96
hier_strict_df_h['y_model-hi-80'] = hier_strict_df_h['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_strict_df_h,
                            Y_df=hier_strict_df, 
                            S=S_strict, 
                            tags=tags_strict,
                            level=[80, 90], 
                            intervals_method='permbu')
total = reconciled.query("unique_id in @tags_grouped['Country/State/Region']").groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.query("unique_id == 'Australia'")[['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# polars
# test methods with PERMBU prediction intervals

# test expect error with grouped structure
# (non strictly hierarchical)
hier_grouped_hat_df_pl = pl.from_pandas(hier_grouped_hat_df)

hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl, [80, 90], 'permbu',)
)

# test PERMBU
hier_strict_df_h_pl = pl.from_pandas(hier_strict_df_h)
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_strict_df_h_pl,
                            Y_df=hier_strict_df_pl, 
                            S=S_strict_pl, 
                            tags=tags_strict_pl,
                            level=[80, 90], 
                            intervals_method='permbu')
total = reconciled.filter(pl.col("unique_id").is_in(tags_grouped['Country/State/Region'])).group_by('ds', maintain_order=True).sum()
pltest.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.filter(pl.col("unique_id") == 'Australia')[['ds', 'y_model/BottomUp']]
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with Bootraped Bootstap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
bootstrap_df = hrec.bootstrap_reconcile(Y_hat_df=hier_grouped_hat_df,
                                        Y_df=hier_grouped_df, S_df=S_grouped_df, tags=tags_grouped,
                                        level=[80, 90],
                                        intervals_method='bootstrap',
                                        num_seeds=2)
assert 'y_model/BottomUp-lo-80' in bootstrap_df.columns
assert 'seed' in bootstrap_df.columns
assert len(set(bootstrap_df["seed"]))==2

In [ ]:
#| hide
# polars
# test methods with Bootraped Bootstap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
bootstrap_df = hrec.bootstrap_reconcile(Y_hat_df=hier_grouped_hat_df_pl,
                                        Y_df=hier_grouped_df_pl, 
                                        S_df=S_grouped_df_pl, 
                                        tags=tags_grouped_pl,
                                        level=[80, 90],
                                        intervals_method='bootstrap',
                                        num_seeds=2)
assert 'y_model/BottomUp-lo-80' in bootstrap_df.columns
assert 'seed' in bootstrap_df.columns
assert len(set(bootstrap_df["seed"]))==2

In [ ]:
#| hide
# test level protection for PERMBU and Normality probabilistic methods
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='Level must be a list containing floating values in the interval [0, 100',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [-1, 80, 90], 'permbu',)
)
test_fail(
    hrec.reconcile,
    contains='Level must be a list containing floating values in the interval [0, 100',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [80, 90, 101], 'normality',)
)

In [ ]:
#| hide
# polars
# test level protection for PERMBU and Normality probabilistic methods
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='Level must be a list containing floating values in the interval [0, 100',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl, [-1, 80, 90], 'permbu',)
)
test_fail(
    hrec.reconcile,
    contains='Level must be a list containing floating values in the interval [0, 100',
    args=(hier_grouped_hat_df_pl, S_grouped_df_pl, tags_grouped_pl, hier_grouped_df_pl, [80, 90, 101], 'normality',)
)

# Example

In [ ]:
#| eval: false
import numpy as np
import pandas as pd

from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.evaluation import HierarchicalEvaluation

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

# Load TourismSmall dataset
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# Create hierarchical seires based on geographic levels and purpose
# And Convert quarterly ds string to pd.datetime format
hierarchy_levels = [['Country'],
                    ['Country', 'State'], 
                    ['Country', 'Purpose'], 
                    ['Country', 'State', 'Region'], 
                    ['Country', 'State', 'Purpose'], 
                    ['Country', 'State', 'Region', 'Purpose']]

Y_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)
qs = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()

# Split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq='QS', n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

# Reconcile the base predictions
reconcilers = [
                BottomUp(),
               MinTrace(method='mint_shrink'),
               ]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)

# Evaluate
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

evaluator = HierarchicalEvaluation(evaluators=[mase])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation = evaluation.set_index(['level', 'metric'])
evaluation = evaluation.map('{:.2f}'.format)

evaluation